In [11]:
class data:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
args = data()
args.filename = "paultan.csv"
args.stemmer = "lemma"
args.dictionary = "automotive"
args.model = "dtm"

In [12]:
# -*- coding: utf-8 -*-
import numpy as np
import lda
import lda.datasets
import csv
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from gensim import corpora, models

from stemming.porter2 import stem
from nltk.stem import *
import unicodecsv
import re
import pyLDAvis.gensim
import gensim

_digits = re.compile('\d')
def contains_digits(d):
    return bool(_digits.search(d))

import enchant
d = enchant.Dict("en_US")
# Or using the /usr/share/dict/british-english word list
with open(args.dictionary + "-english") as word_file:
  english_words = set(word.strip().lower() for word in word_file)
  print(english_words)
  def is_english_word(word):
    return word.lower() in english_words

def process_tokens(tokens,stemmer):
  tokens = [i for i in tokens if not i in en_stop and not contains_digits(i) and is_english_word(i)]
  if stemmer == 'porter':
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(i) for i in tokens]
  elif stemmer == 'porter2':
    tokens = [stem(i) for i in tokens]
  elif stemmer == 'lemma':
    lemmatiser = WordNetLemmatizer()
    tokens = [lemmatiser.lemmatize(i) for i in tokens]
  return tokens


if args.filename == "sample":
  X = lda.datasets.load_reuters()
  dictionary = lda.datasets.load_reuters_vocab()
  titles = lda.datasets.load_reuters_titles()
else:
  f = open(args.filename)
  reader = unicodecsv.reader(f, encoding='utf-8')
  # csv_length = sum(1 for row in reader)
  # f.seek(0) #reset reader position
  identifiers = reader.next()
  contents_idx = identifiers.index("contents")
  title_idx = identifiers.index("title")

  contents = [ row[contents_idx] for row in reader if row[contents_idx] ]

  f.seek(0)
  reader.next()
  titles = [ row[title_idx] for row in reader if row[contents_idx] ]



  texts = list()
  tokenizer = RegexpTokenizer(r'\w+')
  en_stop = get_stop_words('en')
  for idx,i in enumerate(contents):
#     if not idx % 10:
#       print "INFO: Tokenizing articles <{}> ".format(idx)
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)
    texts.append(process_tokens(tokens, args.stemmer))
    # print idx
    # add tokens to list

  print "[DEBUG] Length of Texts : {}".format(len(texts))
  dictionary = corpora.Dictionary(texts)
  corpus = [dictionary.doc2bow(text) for text in texts]


set(['changfeng motor', 'hi-test', 'magnetic', 'alternatives', 'dynasty', 'obstruction', 'bonneville salt flats', 'query', 'loyalty', 'adjustment', 'integrity', 'gonow', 'synthetic', 'saleswoman', 'disposal', 'technique', 'all-wheel drive', 'corrosive', 'jalopy', 'jianghuai', 'safety', 'infant seats', 'gravel', 'roadway', 'protective', 'lessons', 'advantage', 'dynamic', 'activation', 'aston martin', 'faw', 'mph', 'liability', 'balancing', 'drinking', 'lubrication', 'courageous', 'condition', 'tickets', 'retrial', 'transmission', 'race', 'jiangnan', 'recall', 'quick', 'inspection', 'accumulation', 'triumph', 'revolution', 'occurrence', 'corvette', 'four door', 'horn', 'tires', 'cost', 'design', 'enhancement', 'rigorous', 'runner-up', 'wings', 'hongqi', 'wreck', 'fine', 'specification', 'overload', 'sequential', 'optional', 'roewe', 'international', 'changhe', 'appeal', 'racing', 'ignition', 'impact', 'zhongyu', 'jiangling', 'component', 'chassis', 'lubricant', 'hwanghai', 'pedals', 'mac

In [14]:
pyLDAvis.enable_notebook()
model_filename = args.filename.split('.')[0] + '.model'
try:
  ldamodel = models.LdaModel.load(model_filename)
except IOError:
  if(args.model == "lda"):
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=20)
  elif(args.model == "dtm"):
    my_timeslices = [500,500,500,500,500,346]
    ldamodel = gensim.models.wrappers.DtmModel('/Users/Hii/Projects/news_scraper/dtm-darwin64', corpus, my_timeslices, num_topics=20, id2word=dictionary,initialize_lda=True)
  else:
    raise ValueError('Unknown Model Type')
  ldamodel.save(model_filename)
    



In [17]:
  for idx, topic in enumerate(ldamodel.show_topics(topics=10, topn=10, log=False, formatted=False)):
    print "Topic #" + str(idx) + " :",
    for word in topic:
      print word[1],
    print

Topic #0 : year safety increase price car impact point vehicle test cost
Topic #1 : volkswagen perodua year recall scandal brand vehicle automaker model new
Topic #2 : audi drive cockpit new navigation lot comfort map showroom information
Topic #3 : car porsche speed power engine drive weight wheel performance chassis
Topic #4 : design ferrari fiat bentley jeep car year feature new dashboard
Topic #5 : engine speed new transmission automatic torque manual cylinder model mitsubishi
Topic #6 : toyota vehicle new accessory year car part technical passenger fuel
Topic #7 : engine fuel new power efficiency performance oil consumption cylinder technology
Topic #8 : new car model sedan year design debut feature make engine
Topic #9 : control feature new speed display drive safety leather automatic model
Topic #10 : year safety increase price car impact point vehicle test cost
Topic #11 : volkswagen perodua year recall scandal brand vehicle automaker model new
Topic #12 : audi drive cockpit ne

In [10]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=20)
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

/Users/Hii/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/Users/Hii/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/Users/Hii/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:672: DeprecationWarning: PlainTextFormatter._type_printers_default is deprecated: use @default decorator instead.
  def _type_printers_default(self):
/Users/Hii/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
9      31.608107        1       1 -0.135226  0.078856
7      11.592118        1       2 -0.022760  0.047250
8      11.083459        1       3  0.039034  0.089596
2      10.943581        1       4  0.087372  0.040781
5       7.136189        1       5  0.289445 -0.165515
3       6.554779        1       6 -0.187549 -0.058032
0       5.796413        1       7 -0.134096 -0.236990
4       5.737764        1       8  0.064627  0.056495
6       4.997507        1       9  0.033564  0.166960
1       4.550082        1      10 -0.034411 -0.019402, topic_info=    Category        Freq               Term  Total  loglift  logprob
108  Default  190.000000             camera    190  30.0000  30.0000
19   Default   83.000000             window     83  29.0000  29.0000
72   Default   65.000000           internet     65  28.0000  28.0000
9    Default   52.000000           keyboard     52  27.0000  27.0000
159  Default   67.000000               data     67  26.0000  26.0000
129  Default  103.000000            display    103  25.0000  25.0000
92   Default  104.000000          processor    104  24.0000  24.0000
112  Default   30.000000             medium     30  23.0000  23.0000
110  Default   50.000000              store     50  22.0000  22.0000
73   Default  216.000000                new    216  21.0000  21.0000
78   Default   95.000000             screen     95  20.0000  20.0000
149  Default   27.000000        performance     27  19.0000  19.0000
100  Default   63.000000             market     63  18.0000  18.0000
114  Default   25.000000           wireless     25  17.0000  17.0000
117  Default   22.000000               ipad     22  16.0000  16.0000
156  Default   44.000000             update     44  15.0000  15.0000
18   Default  179.000000             source    179  14.0000  14.0000
103  Default   25.000000            quality     25  13.0000  13.0000
35   Default   30.000000               word     30  12.0000  12.0000
56   Default   62.000000              video     62  11.0000  11.0000
54   Default   18.000000               rate     18  10.0000  10.0000
107  Default   42.000000             system     42   9.0000   9.0000
133  Default   37.000000              share     37   8.0000   8.0000
53   Default   48.000000               zoom     48   7.0000   7.0000
137  Default   37.000000         technology     37   6.0000   6.0000
128  Default   52.000000             design     52   5.0000   5.0000
11   Default   23.000000              speed     23   4.0000   4.0000
69   Default   31.000000             access     31   3.0000   3.0000
93   Default   33.000000           software     33   2.0000   2.0000
139  Default   56.000000        information     56   1.0000   1.0000
..       ...         ...                ...    ...      ...      ...
99   Topic10    0.050264         electronic      1   0.7379  -7.2076
21   Topic10    0.050343          integrate      1   0.7379  -7.2077
38   Topic10    0.050261       disadvantage      1   0.7379  -7.2077
155  Topic10    0.050345             symbol      1   0.7379  -7.2077
57   Topic10    0.050261        enhancement      1   0.7379  -7.2077
67   Topic10    0.051429           audience      1   0.7379  -7.2077
61   Topic10    0.050653              youth      1   0.7379  -7.2077
42   Topic10    0.050345        informative      1   0.7379  -7.2077
25   Topic10    0.050086         innovation      1   0.7379  -7.2077
3    Topic10    0.051429            gradual      1   0.7379  -7.2077
15   Topic10    0.050149              exact      1   0.7379  -7.2077
77   Topic10    0.050149  telecommunication      1   0.7379  -7.2077
120  Topic10    0.050136            graphic      1   0.7379  -7.2077
39   Topic10    3.354846               term     35   0.7108  -3.6794
64   Topic10    1.015075          broadband     12   0.6507  -4.8100
18   Topic10   13.534090             source    179   0.4501  -2.30